# Demonstrate ``nscan``

In this example, we demonstrate the `apstools.plans.nscan()` plan.  The `nscan()` plan scans over ``n`` variables moved together, each in equally spaced steps.

Here, we scan two motors together, each in equally spaced steps.  We use an [`swait`](https://htmlpreview.github.io/?https://raw.githubusercontent.com/epics-modules/calc/R3-6-1/documentation/swaitRecord.html) record (part of the *userCalc* support from synApps) as a detector.  We configure the `swait` record with a calculation (from `apstools.plans.swait_setup_random_number()`) that computes a noisy (random number) signal.  

For this demo, we do not need the databroker since we do not plan to review any of this data after collection.  We'll display the data during the scan using the *LiveTable()* code.

In [1]:
from apstools.synApps import SwaitRecord, setup_random_number_swait
from apstools.plans import nscan
from bluesky import RunEngine
from bluesky.callbacks import LiveTable
from ophyd import EpicsMotor

RE = RunEngine({})

Set the prefix for the EPICS IOC that provides the PVs we'll use here.

In [2]:
IOC = "gp:"

Connect to our motors and create the *noisy* detector.

In [3]:
m1 = EpicsMotor(f"{IOC}m1", name="m1")
m2 = EpicsMotor(f"{IOC}m2", name="m2")
noisy = SwaitRecord(f"{IOC}userCalc1", name="noisy")

m1.wait_for_connection()
m2.wait_for_connection()
noisy.wait_for_connection()

# configure the *detector* as a random number generator using a calculation.
setup_random_number_swait(noisy)

Measure the noisy detector while step scanning both the m1 & m2 motors together.  We'll move m2 twice as far as m1, like a $\theta$-$2\theta$ scan on a diffractometer.

In [4]:
RE(
    nscan([noisy, ], m1, 2, 0, m2, 4, 0, num=6),
    LiveTable(["m1", "m2", "noisy_val"])
    )



+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |         m2 |
+-----------+------------+------------+------------+
|         1 | 16:09:07.5 |    2.00000 |    4.00000 |
|         2 | 16:09:08.6 |    1.60000 |    3.20000 |
|         3 | 16:09:09.7 |    1.20000 |    2.40000 |
|         4 | 16:09:10.8 |    0.80000 |    1.60000 |
|         5 | 16:09:11.9 |    0.40000 |    0.80000 |
|         6 | 16:09:13.0 |    0.00000 |    0.00000 |
+-----------+------------+------------+------------+
generator nscan ['2cdf0415'] (scan num: 1)




('2cdf0415-5ab1-4d9c-9622-9ece1131881b',)